In [ ]:
!pip install -q transformers datasets numba

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda
from transformers import pipeline

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("drive/MyDrive/5_products_dataset_preprocessed.csv")
df

,product_id,review_body,review_headline,star_rating,helpful_votes,total_votes,review_date,review_year
0,B003L1ZYYM,job,Five Stars,5,0,0,2015-08-31,2015
1,B0001FTVEK,great product,Five Stars,5,0,0,2015-08-31,2015
2,B0012S4APK,great product fair price easy install worked f...,Great product at a fair price,5,0,0,2015-08-31,2015
3,B003EM8008,probably best earphones ever better apple ones...,Good quality head phones!,5,0,0,2015-08-31,2015
4,B0001FTVEK,work great sounds amazing,Four Stars,4,0,0,2015-08-31,2015
...,...,...,...,...,...,...,...,...
54385,B0001FTVEK,headphones months ca n't believe life changed ...,Can't understand the negative reviews!,4,134,148,2004-07-31,2004
54386,B0002L5R78,tried quite cables past months hd setup far be...,Best HDMI Cable,5,10,18,2004-07-12,2004
54387,B0001FTVEK,first set wireless headphones 've looking set ...,"OK headphones, but not what I was looking for...",3,27,30,2004-07-07,2004
54388,B0001FTVEK,lived europe 2003 purchased pair sennheiser wi...,Extremely disappointed,1,311,360,2004-07-07,2004


In [ ]:
def get_batches(reviews, sample):

  input = "summarize:"
  count = 0
  checkpoint = 0


  #batch creation
  #creo un array di testi concatenati, ognuno dei quali ha massimo 1024 token
  #lo stesso procedimento verrà fatto con i riassunti generati!
  # si parte da checkpoint in poi, fino a finire il ciclo.

  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < 1024:
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " . "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      #print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      #print("Number of reviews processed: {}".format(i))
      input = "summarize: " + reviews[i] + " . "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    #print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    #print("Number of reviews processed: {}".format(i))

  return batches

In [ ]:
def summarize_final(df):
  final_sum = []
  ids = df['product_id'].unique()
  years = df['review_year'].unique()
  for year in years:
      part = df[(df["product_id"] == "B0002L5R78") & (df["review_year"] == year)]
      reviews = part["review_body"].tolist()
      batches = get_batches(reviews, False)

      summaries_first_stage = []
      for input in batches:
        inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
        output = model.generate(inputs, max_length=100)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        # print(decoded_output)
        summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

      input = "summarize: "
      count = 0
      checkpoint = 0
      batches_second_stage = []
      i = 0
      for i in range(0,len(summaries_first_stage)):
        if count + len(summaries_first_stage[i].split(" ")) < 1024:
          #print(cleaned_reviews[i])
          input = input + summaries_first_stage[i] + " . "
          #print(input)
          count = count + len(summaries_first_stage[i].split(" "))
        else:
          batches_second_stage.append(input)
          #print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
          #print("Number of reviews processed: {}".format(i))
          input = "summarize: " + summaries_first_stage[i] + " . "
          count = len(summaries_first_stage[i].split(" "))

      if(count != 0):
        batches_second_stage.append(input)
        #print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
        #print("Number of reviews processed: {}".format(i))

      summaries_second_stage = []
      for input in batches_second_stage:
        inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
        output = model.generate(inputs, max_length=100)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

      print("Product " + str(id) + ", year " + str(year) + ": " + str(summaries_second_stage))
      if(len(summaries_second_stage) != 0):
        final_sum.append(summaries_second_stage)
  return final_sum

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-xsum')
model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-xsum')

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-xsum.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
summaries = summarize_final(df)

Product <built-in function id>, year 2015: ['A look at some of the best cable reviews for the ps3 video game console.']
Product <built-in function id>, year 2014: ['As part of our series of product reviews, we look at some of the best hdmi cables.']
Product <built-in function id>, year 2013: ['As part of our series of product reviews, we take a look at some of the best cables we have ever used.']
Product <built-in function id>, year 2012: ['Here are some of the best and worst hdmi cable products.']
Product <built-in function id>, year 2011: ["If you're looking for a cheap way to hook up your home theatre system or dvd player to your computer, try these hdmi cables."]
Product <built-in function id>, year 2010: ["As part of our series on the best cable products, we've been looking at some of the best hdmi cable products."]
Product <built-in function id>, year 2009: ["Here's a round-up of the best and worst hdmi cable reviews from around the world."]
Product <built-in function id>, year 2

In [ ]:
ds = df.groupby(by=['product_id', 'review_year'], sort=True).mean()
ds = ds.sort_values(by=['product_id', 'review_year'], ascending=[True, True])
ds = ds['star_rating'].tolist()
ds

<ipython-input-15-7f76ac37e078>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds = df.groupby(by=['product_id', 'review_year'], sort=True).mean()


product_id  review_year
B0001FTVEK  2004           3.461538
            2005           4.238095
            2006           3.765625
            2007           4.076087
            2008           4.056291
            2009           3.926625
            2010           3.943590
            2011           3.907668
            2012           3.985965
            2013           4.040913
            2014           4.093432
            2015           4.068258
B0002L5R78  2004           5.000000
            2005           1.000000
            2006           4.875000
            2007           4.580769
            2008           4.566857
            2009           4.581467
            2010           4.489910
            2011           4.244107
            2012           4.483405
            2013           4.448029
            2014           4.482980
            2015           4.549091
B0012S4APK  2008           4.000000
            2009           4.554217
            2010           4.518160
    

In [ ]:
#Sentiment analysis
#Bert-base-multilingual-uncased-sentiment is a model fine-tuned for sentiment analysis on product reviews in six languages: English, Dutch, German, French, Spanish and Italian.
pipe = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
i = 0
for sum in summaries:
  val = pipe(str(sum))
  print("Second Stage: " + str(val[0].get("score")))
  print("Expected Rating: " + str(ds[i]/5))

In [ ]:
val = pipe("As part of our series on the best headphones, we've been looking at some of the best earbuds.")
print("Second Stage: " + str(val))
print("Expected Rating: " + str(4.381714/5))

Second Stage: [{'label': '5 stars', 'score': 0.6588751077651978}]
Expected Rating: 0.8763428


In [ ]:
'As part of our series on the best cable products, we take a look at some of the best hdmi cable products on the market.'
val = pipe('As part of our series on the best cable products, we take a look at some of the best hdmi cable products on the market.')
print("Second Stage: " + str(val))
print("Expected Rating: " + str(4.699761/5))

Second Stage: [{'label': '5 stars', 'score': 0.757246196269989}]
Expected Rating: 0.9399521999999999


In [ ]:
val = pipe('Here are some of the best and worst-selling wireless headphones available on the market.')
print("Second Stage: " + str(val))
print("Expected Rating: " + str(3.926625/5))

Second Stage: [{'label': '5 stars', 'score': 0.771148145198822}]
Expected Rating: 0.785325


In [ ]:
val = pipe('A cheetah mount has been rated "excellent" by the US Consumer Product Safety Commission (CPSC).')
print("Second Stage: " + str(val))
print("Expected Rating: " + str(4.000000/5))

Second Stage: [{'label': '5 stars', 'score': 0.5100550055503845}]
Expected Rating: 0.8


In [ ]:
val = pipe('As part of our series of product reviews, we take a look at some of the best cables we have ever used.')
print("Second Stage: " + str(val))
print("Expected Rating: " + str(4.448029/5))

Second Stage: [{'label': '5 stars', 'score': 0.6844561100006104}]
Expected Rating: 0.8896058


In [ ]:
val = pipe("i hate this product, its so full of defects and i will never buy it again")
print("Second Stage: " + str(val))

Second Stage: [{'label': '1 star', 'score': 0.9292482137680054}]


In [ ]:
val = pipe("Hdmi cable is one of the best buys ever made.")
print("Second Stage: " + str(val))

Second Stage: [{'label': '5 stars', 'score': 0.9593029022216797}]


Product <built-in function id>, year 2015: ['A look at some of the best cable reviews for the ps3 video game console.']

Product <built-in function id>, year 2014: ['As part of our series of product reviews, we look at some of the best hdmi cables.']


Product <built-in function id>, year 2013: ['As part of our series of product reviews, we take a look at some of the best cables we have ever used.']


Product <built-in function id>, year 2012: ['Here are some of the best and worst hdmi cable products.']


Product <built-in function id>, year 2011: ["If you're looking for a cheap way to hook up your home theatre system or dvd player to your computer, try these hdmi cables."]


Product <built-in function id>, year 2010: ["As part of our series on the best cable products, we've been looking at some of the best hdmi cable products."]


Product <built-in function id>, year 2009: ["Here's a round-up of the best and worst hdmi cable reviews from around the world."]


Product <built-in function id>, year 2008: ["As part of our series on the best cable products, we've been looking at some of the most popular hdmi cables on the market."]


Product <built-in function id>, year 2007: ['What are the best and worst hdmi cables?']


Product <built-in function id>, year 2006: ["Here's a quick rundown of some of the things I've learned over the past few days:."]


Product <built-in function id>, year 2005: ['The BBC has published a list of the best and worst cable brands in the UK.']


Product <built-in function id>, year 2004: ["I've been trying to find the best cable to connect my laptop to my TV and this is it."]